# Задача минимум

In [1]:
import pandas as pd
import numpy as np

In [2]:
has_cancer = 'p_cncr'
has_no_cancer = 'p_cntrl'

In [3]:
df = pd.read_csv('data.csv')

In [4]:
df

,Unnamed: 0,p_cntrl4224,p_cncr8407,p_cntrl6764,p_cncr3273,p_cntrl1262,p_cncr8861,p_cncr5984,p_cncr8964,p_cncr7142,...,p_cntrl3871,p_cntrl9527,p_cncr8117,p_cntrl3797,p_cncr2900,p_cncr4395,p_cncr2896,p_cncr3688,p_cncr3488,p_cncr5857
0,10100011111111100001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001000100110001111,16,22238,2429,205,211477,235,4040,89,3297,...,810,569497,90,2639,5750326,1664372,57,12861,2803,5306
2,11100110010111011110,200306,1883652,75266,137451,33655,19243,210799,75,21905373,...,29868,16047,102685,2136875,372,20168,9549,907939,2165,124579
3,1001000101000110111,3589,5246,44866,4296,448249,390585,3171,13306829,410,...,55173,350512,67439,23863,24240,849656,480214,13620,2346351,210529
4,10101011001001001011,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4984,100111010101,1084,92,8425,5749,173909,686,9872,184,3017,...,216,369164,3271,9210,1580975,4131,4592,126,8771,4882
4985,1000100001011011010,154963,119032,114491,34791,30286,210002,119774,3188,41409,...,124886,157290,1183759,498661,1303,1415342,561366,73925,4206,113270
4986,11111000011101001001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4987,1101000101101111110,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# переведем в строку
DNAs = df['Unnamed: 0'].values.astype(str)

In [6]:
# теперь нарастим данные
dnas = []
for dna in DNAs:
    if len(dna) < 20:
        dnas.append('0'*(20 - len(dna)) + dna)
    else:
        dnas.append(dna)
        
dnas = np.array(dnas)

In [7]:
def init_dict(name: str) -> dict:
    data_dict = {}
    if has_cancer in name:
        cancer = True
    elif has_no_cancer in name:
        cancer = False
    data_dict = {'name': name, 'has_cancer': cancer}
    
    return data_dict

In [20]:
def make_methylated_dict(input_dict: dict, DNA_types: list, frequency: np.array, non_methylated=False) -> dict:
    
    data_dict = input_dict.copy()
    
    for i in range(20):
        count_0 = 0
        count_1 = 0
        for _, (dna_seq, freq) in enumerate(zip(DNA_types, frequency)):
            if dna_seq[i] == '1':
                count_1 += freq
            elif dna_seq[i] == '0':
                count_0 += freq
                
        summ = count_0 + count_1
        if non_methylated:
            data_dict[f'{i}_idx_0'] = count_0 / summ
        data_dict[f'{i}_idx_1'] = count_1 / summ
        
    return data_dict        

In [21]:
def make_patterns_freq_dict(input_dict: dict, DNA_types: list, frequency: np.array) -> dict:
    data_dict = input_dict.copy()
    
    for i in range(19):
        count_11 = 0
        count_10 = 0
        count_01 = 0
        count_00 = 0
        
        for _, (dna_seq, freq) in enumerate(zip(DNA_types, frequency)):
            if dna_seq[i:i+2] == '11':
                count_11 += freq
            elif dna_seq[i:i+2] == '10':
                count_10 += freq
            elif dna_seq[i:i+2] == '01':
                count_01 += freq
            elif dna_seq[i:i+2] == '00':
                count_00 += freq
            
        data_dict[f'{i}_{i+1}_patter_00'] = count_00
        data_dict[f'{i}_{i+1}_patter_10'] = count_10
        data_dict[f'{i}_{i+1}_patter_01'] = count_01
        data_dict[f'{i}_{i+1}_patter_11'] = count_11

    return data_dict

In [22]:
def log_dict(input_dict: dict) -> dict:
    data_dict = input_dict.copy()
    keys = data_dict.keys()
    for key in keys:
        if type(data_dict[key]) != str and type(data_dict[key]) != bool:
            data_dict[key] = np.log(np.array(data_dict[key]))     
        
    return data_dict

In [23]:
dataset = df.copy().drop(columns = 'Unnamed: 0')
dataset

,p_cntrl4224,p_cncr8407,p_cntrl6764,p_cncr3273,p_cntrl1262,p_cncr8861,p_cncr5984,p_cncr8964,p_cncr7142,p_cncr1466,...,p_cntrl3871,p_cntrl9527,p_cncr8117,p_cntrl3797,p_cncr2900,p_cncr4395,p_cncr2896,p_cncr3688,p_cncr3488,p_cncr5857
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,16,22238,2429,205,211477,235,4040,89,3297,28083,...,810,569497,90,2639,5750326,1664372,57,12861,2803,5306
2,200306,1883652,75266,137451,33655,19243,210799,75,21905373,209856,...,29868,16047,102685,2136875,372,20168,9549,907939,2165,124579
3,3589,5246,44866,4296,448249,390585,3171,13306829,410,5434,...,55173,350512,67439,23863,24240,849656,480214,13620,2346351,210529
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4984,1084,92,8425,5749,173909,686,9872,184,3017,22596,...,216,369164,3271,9210,1580975,4131,4592,126,8771,4882
4985,154963,119032,114491,34791,30286,210002,119774,3188,41409,7078,...,124886,157290,1183759,498661,1303,1415342,561366,73925,4206,113270
4986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4987,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
def make_patients_df(DNA_types: list, df: pd.DataFrame, non_methylated=False):
    initial_df = df.copy()  # .drop('Unnamed: 0')
    data_dicts = []
    for key in initial_df.keys():
        name = key
        frequency = initial_df[key].values
        data_dict = init_dict(name=name)
        data_dict = make_methylated_dict(data_dict, DNA_types, frequency, non_methylated)
        data_dict = make_patterns_freq_dict(data_dict, DNA_types, frequency)
        data_dict = log_dict(data_dict)
        data_dicts.append(data_dict)
    
        
    df = pd.DataFrame(data = data_dicts)
    
    return df, data_dict

In [25]:
_df, data_dict = make_patients_df(dnas, dataset)  # ['p_cntrl4224'], 'p_cntrl4224')
_df

,name,has_cancer,0_idx_1,1_idx_1,2_idx_1,3_idx_1,4_idx_1,5_idx_1,6_idx_1,7_idx_1,...,16_17_patter_01,16_17_patter_11,17_18_patter_00,17_18_patter_10,17_18_patter_01,17_18_patter_11,18_19_patter_00,18_19_patter_10,18_19_patter_01,18_19_patter_11
0,p_cntrl4224,False,-0.209753,-0.682014,-0.481578,-0.348247,-0.812759,-0.065296,-0.621512,-1.520391,...,20.757955,20.952918,20.446439,21.465594,18.798016,19.076332,21.365855,19.232995,20.679950,18.544378
1,p_cncr8407,True,-0.297776,-0.147430,-0.809350,-0.254466,-0.319309,-0.103592,-0.385879,-1.996438,...,17.930703,20.032682,20.855742,19.461682,21.513473,19.447947,20.445357,21.168206,20.319036,20.642921
2,p_cntrl6764,False,-0.387062,-0.450260,-1.984531,-1.487168,-1.058070,-0.323872,-2.027257,-0.093262,...,21.041763,20.267854,20.807322,21.099658,19.731720,20.129471,21.204803,20.048143,20.646544,19.841707
3,p_cncr3273,True,-0.111078,-1.915340,-0.620267,-0.478637,-0.919630,-0.066536,-1.062583,-1.100944,...,21.232418,19.947723,21.065020,21.058353,20.371180,20.403433,21.439382,20.492739,20.447523,20.269727
4,p_cntrl1262,False,-0.191102,-1.211476,-3.788013,-0.735700,-0.498540,-2.000105,-0.801118,-0.536222,...,22.047408,20.235941,18.240764,20.102936,20.274567,22.067555,19.209734,21.220659,19.809895,21.763362
5,p_cncr8861,True,-1.204066,-0.660623,-1.685986,-0.537415,-0.811426,-0.571365,-0.133072,-0.500184,...,18.842604,20.545335,21.282594,19.905169,20.758844,20.122185,19.639898,19.463054,21.339701,20.986279
6,p_cncr5984,True,-0.079105,-0.516185,-1.975311,-0.550035,-1.852444,-0.739608,-0.582920,-0.658946,...,20.096836,20.465327,20.236464,19.483282,19.993729,20.740861,20.505336,21.015318,18.418637,18.895109
7,p_cncr8964,True,-1.038401,-0.445408,-0.685371,-0.186944,-0.604294,-0.272086,-1.990773,-0.229019,...,21.516382,19.440085,19.966976,21.117072,20.233219,20.728068,20.006153,20.102826,21.104341,20.799952
8,p_cncr7142,True,-0.647025,-1.204416,-0.231438,-0.851404,-0.267306,-0.049706,-1.254216,-0.910299,...,19.996210,21.028173,18.306162,19.633014,19.690108,21.131193,19.426611,20.684732,18.838511,20.614989
9,p_cncr1466,True,-0.723325,-1.687661,-1.289578,-0.561997,-0.225809,-0.136736,-0.538029,-0.795957,...,20.088457,20.097091,20.231569,20.243266,19.870158,19.915576,19.880011,19.710167,20.500202,20.047729


# Основное задание

# Machine Learning

In [26]:
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt


In [33]:
trainY = np.where(_df['has_cancer'].values, 1, 0)
trainY
trainX = _df.copy().drop(columns=['name', 'has_cancer']).values
print(trainX.shape)
type(trainX)

(30, 96)


numpy.ndarray

In [37]:
lgr = LogisticRegression(max_iter=1000)
lgr.fit(trainX[1:], trainY[1:])

LogisticRegression(max_iter=1000)

In [51]:
lgr.predict_proba([trainX[1]])

array([[0.00177541, 0.99822459]])

In [50]:
np.argmax(lgr.predict_proba([trainX[1]]))

1

In [42]:
trainY[1]

1

In [46]:
np.concatenate((trainX[0:1], trainX[1:2]), axis=0)

array([[-0.20975272, -0.68201449, -0.48157769, -0.34824703, -0.81275897,
        -0.06529627, -0.62151155, -1.52039052, -0.13783958, -1.23451521,
        -0.09620499, -1.28541979, -0.27849802, -0.10848478, -0.76558189,
        -0.57584005, -0.58569928, -0.33230297, -2.2456579 , -1.09398072,
        19.28407129, 21.0187122 , 19.72330632, 20.94540556, 20.24360241,
        20.2175301 , 20.68437217, 20.73675294, 19.60041593, 20.23705281,
        20.61423433, 21.03110389, 20.06533268, 20.95521551, 19.86141914,
        20.71938487, 18.56650622, 18.2746601 , 21.2320559 , 21.01001792,
        18.30600277, 21.05317972, 18.54243887, 21.19609241, 20.96960367,
        20.92137729, 19.11723203, 20.02685918, 19.62766127, 18.16418344,
        21.49526696, 20.24793475, 19.3815494 , 21.41925775, 18.8282537 ,
        20.47489957, 15.61968761, 19.47570797, 21.53908152, 20.2819435 ,
        19.11677066, 21.47122343, 18.34478075, 20.48947063, 20.07266611,
        19.35878216, 21.3063399 , 20.25916829, 18.4

# Leave-one_out cross validation

In [52]:
model = None
total = 0
correct = 0
for i in range(len(trainY)):
    model = LogisticRegression(max_iter=1000)
    model.fit(np.concatenate((trainX[0:i], trainX[i+1:]), axis=0), np.concatenate((trainY[0:i], trainY[i+1:]), axis=0))
    prediction = np.argmax(model.predict_proba([trainX[i]]))
    total += 1
    if prediction == trainY[i]:
        correct += 1
    print(f'iter: {i}  Predicted Label: {prediction}  True Label: {trainY[i]}')
    
print(correct / total)

iter: 0  Predicted Label: 1  True Label: 0
iter: 1  Predicted Label: 1  True Label: 1
iter: 2  Predicted Label: 0  True Label: 0
iter: 3  Predicted Label: 1  True Label: 1
iter: 4  Predicted Label: 1  True Label: 0
iter: 5  Predicted Label: 0  True Label: 1
iter: 6  Predicted Label: 0  True Label: 1
iter: 7  Predicted Label: 1  True Label: 1
iter: 8  Predicted Label: 1  True Label: 1
iter: 9  Predicted Label: 1  True Label: 1
iter: 10  Predicted Label: 1  True Label: 0
iter: 11  Predicted Label: 1  True Label: 1
iter: 12  Predicted Label: 1  True Label: 1
iter: 13  Predicted Label: 1  True Label: 1
iter: 14  Predicted Label: 1  True Label: 0
iter: 15  Predicted Label: 0  True Label: 1
iter: 16  Predicted Label: 1  True Label: 0
iter: 17  Predicted Label: 1  True Label: 0
iter: 18  Predicted Label: 1  True Label: 1
iter: 19  Predicted Label: 1  True Label: 1
iter: 20  Predicted Label: 0  True Label: 0
iter: 21  Predicted Label: 1  True Label: 0
iter: 22  Predicted Label: 1  True Label: 

# Multiple threading

In [55]:
import time
treads_num = 1

In [56]:
# OpenMP: number of parallel threads.
%env OMP_NUM_THREADS=1

env: OMP_NUM_THREADS=1


In [58]:
%env OMP_NUM_THREADS=1
since = time.time()
for i in range(len(trainY)):
    model = LogisticRegression(max_iter=1000)
    model.fit(np.concatenate((trainX[0:i], trainX[i+1:]), axis=0), np.concatenate((trainY[0:i], trainY[i+1:]), axis=0))
    prediction = np.argmax(model.predict_proba([trainX[i]]))
    total += 1
    if prediction == trainY[i]:
        correct += 1
#     print(f'iter: {i}  Predicted Label: {prediction}  True Label: {trainY[i]}')
    
time_elapsed = time.time() - since
print(correct / total)
print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

env: OMP_NUM_THREADS=1
0.5666666666666667
Training complete in 0m 2s


In [60]:
%env OMP_NUM_THREADS=2
since = time.time()
for i in range(len(trainY)):
    model = LogisticRegression(max_iter=1000)
    model.fit(np.concatenate((trainX[0:i], trainX[i+1:]), axis=0), np.concatenate((trainY[0:i], trainY[i+1:]), axis=0))
    prediction = np.argmax(model.predict_proba([trainX[i]]))
    total += 1
    if prediction == trainY[i]:
        correct += 1
#     print(f'iter: {i}  Predicted Label: {prediction}  True Label: {trainY[i]}')
    
time_elapsed = time.time() - since
print(correct / total)
print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

env: OMP_NUM_THREADS=4
0.5666666666666667
Training complete in 0m 2s


# Дополнительное задание

In [63]:
DNA = 'GTGATTGTGGGCACAGCTACAAAAC1GGGGTTGGATAAGTC2GCTC3GCC4GGGGCC5GAGGGCC6GTCTC7GTGC8GGGGGC9GGGGAAGGGGC10GTG'+\
    'AGGC11GC12GGAGATGG13GAGAAAAC14GCTAACCC15GC16GTTCTTGATGGGAGGCC17GC18GTCC19TGGGAGATGGGGGT20AGC'

In [65]:
len(DNA)

192

 ## Ничего не понятно!